# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다.
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다.
4. A, B와 대응되는 C를 추출한다.
5. D를 입력받는다.

=>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ?] <br>
모델이 예측한 가장 적합한 단어 : 잠입 <br>
당신의 답변과 모델 예측의 유사도 : 0.34 <br>
아쉽네요. 더 생각해보세요.

In [1]:
# 텍스트 전처리

from lxml import etree      # xml, html 파일을 처리하기 위한 경량화된 라이브러리, etree : xml파일을 파싱하고 조작할 수 있는 함수 
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import pandas as pd
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec

In [2]:
# BOBR210001636718.json 파일 경로
file_path = 'data/BOBR210001636718.json'

# JSON 파일 불러오기
with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 현재 json_data의 최상위 키들을 출력해봅니다.
print("JSON 데이터의 최상위 키:", json_data.keys())

# 만약 'text' 키가 없다면, 오류가 발생합니다.
# ['header', 'text', 'SJML'] 와 같이 'text'가 포함되어 있는지 확인하세요.
# 만약 있다면, 오류는 다른 곳에서 발생했을 수 있습니다.

JSON 데이터의 최상위 키: dict_keys(['SJML'])


In [3]:
file_path = './data/BOBR210001636718.json'

with open(file_path, 'r', encoding='utf-8') as f:
    json_data = json.load(f)

sentences = [item['content'] for item in json_data['SJML']['text']]

okt = Okt()
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords
ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocessed_data = []

for sentence in sentences:
    #nouns = okt.morphs(sentence)
    tokens = okt.morphs(sentence, stem=True)
    tokens = [token for token in tokens if token not in ko_stopwords]
    preprocessed_data.append(tokens)

preprocessed_data[:5]

[['이수근', '은', '너무', '화', '내다', '너무', '시끄럽다', ','],
 ['백',
  '번',
  '만',
  '번',
  '돈까스',
  '가격',
  '책정',
  '은',
  '이수근',
  '맞다',
  '봄',
  '.',
  '진짜',
  '분명',
  '서다',
  '400',
  '2만',
  '인데',
  '는',
  '(',
  '반',
  '사회',
  '적용',
  ')',
  '이네',
  '이르다',
  '욕',
  '하다',
  '사람',
  '분명하다',
  '생기다',
  ','],
 ['왤케', '싸우다', '임'],
 ['그냥',
  '소스',
  '조그마하다',
  '그릇',
  '담기다',
  '부어',
  '먹다',
  '찍다',
  '먹다',
  '알다',
  '하다',
  '않다',
  ','],
 ['44만원', '식', '재료', '판매', '수익', '22만원', ',']]

In [4]:
model = Word2Vec(
    sentences=preprocessed_data,
    vector_size=100,
    window=5,
    min_count=5, 
    sg=0
)

model.wv.vectors.shape

(520, 100)

In [5]:
import random

all_words = list(model.wv.index_to_key)
    
while True:
    a_word, b_word = random.sample(all_words, 2)
    similarity = model.wv.similarity(a_word, b_word)

    if similarity >= 0.8:
        print(f'A: {a_word}, B: {b_word}')
        break

A: 맛, B: 비속어


In [ ]:
candidate_c_words = [word for word in all_words if word not in [a_word, b_word]]

c_word = random.choice(candidate_c_words)
analogy_result = model.wv.most_similar(positive=[c_word, a_word], negative=[b_word], topn=1)

predicted_d_word = analogy_result[0][0]

print(f'관계 [{a_word} : {b_word} = {c_word} : ?]')

user_input = input('모델이 예측한 가장 적합한 단어를 입력하세요: ').strip()

print(f'\n모델이 예측한 가장 적합한 단어: {predicted_d_word}')

#if user_input in model.wv.key_to_index:
#    similarity = model.wv.similarity(user_input, predicted_d_word)
#    print(f'당신의 답 {user_input} 과 모델의 예측 유사도 : {similarity:.2f}')
#else:
#    print('아쉽네요. 더 생각해보세요.')

similarity = model.wv.similarity(user_input, predicted_d_word)
print(f'당신의 답 {user_input} 과 모델의 예측 유사도 : {similarity:.2f}')
if similarity < 0.5:
    print('아쉽네요. 더 생각해보세요.')


관계 [맛 : 비속어 = 여유 : ?]

모델이 예측한 가장 적합한 단어: 끄다
아쉽네요. 더 생각해보세요.


---

In [10]:
import os
import json
from konlpy.tag import Okt
from gensim.models import Word2Vec

# 1. 모든 JSON 파일의 경로를 가져오기
file_list = [f for f in os.listdir('data/media') if f.startswith('BOBR') and f.endswith('.json')]
file_paths = ['data/media/' + file for file in file_list]

In [11]:
# 2. 모든 파일의 데이터를 하나의 리스트로 통합
all_sentences = []
for file_path in file_paths:
    with open(file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    # JSON 파일 구조에 맞춰 'content' 추출
    sentences = [item['content'] for item in json_data['SJML']['text']]
    all_sentences.extend(sentences)

In [ ]:
# 3. 데이터 전처리
okt = Okt()

def preprocess_text(text):
    # 명사, 동사, 형용사만 추출하도록 정의
    try:
        nouns = okt.nouns(text)
        verbs = [word[0] for word in okt.pos(text) if word[1] == 'Verb']
        adjectives = [word[0] for word in okt.pos(text) if word[1] == 'Adjective']
        return nouns + verbs + adjectives
    except Exception:
        return []

processed_sentences = [preprocess_text(sentence) for sentence in all_sentences if isinstance(sentence, str) and len(sentence) > 0]

print(f"총 {len(file_list)}개의 파일에서 {len(all_sentences)}개의 문장을 불러왔습니다.")
print(f"전처리 완료. 총 {len(processed_sentences)}개의 문장이 학습에 사용됩니다.")

In [ ]:
# 4. Word2Vec 모델 학습
model = Word2Vec(
    sentences=processed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=0
)

print("\n대량 데이터로 모델 학습 완료!")

KeyboardInterrupt: 